<a href="https://colab.research.google.com/github/nehan0306/CodeClause_Churn_Prediction/blob/main/CodeClause_Churn_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving churn-bigml-80.csv to churn-bigml-80.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
import seaborn as sns

In [3]:
data = pd.read_csv(r"churn-bigml-80.csv")

In [4]:
data.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int64  
 2   Area code               2666 non-null   int64  
 3   International plan      2666 non-null   object 
 4   Voice mail plan         2666 non-null   object 
 5   Number vmail messages   2666 non-null   int64  
 6   Total day minutes       2666 non-null   float64
 7   Total day calls         2666 non-null   int64  
 8   Total day charge        2666 non-null   float64
 9   Total eve minutes       2666 non-null   float64
 10  Total eve calls         2666 non-null   int64  
 11  Total eve charge        2666 non-null   float64
 12  Total night minutes     2666 non-null   float64
 13  Total night calls       2666 non-null   int64  
 14  Total night charge      2666 non-null   

In [6]:
selector = SelectKBest(score_func = chi2, k=9)

In [7]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [8]:
data['Churn'] = encoder.fit_transform(data['Churn'])

In [9]:
columns = data.columns.tolist()
columns

['State',
 'Account length',
 'Area code',
 'International plan',
 'Voice mail plan',
 'Number vmail messages',
 'Total day minutes',
 'Total day calls',
 'Total day charge',
 'Total eve minutes',
 'Total eve calls',
 'Total eve charge',
 'Total night minutes',
 'Total night calls',
 'Total night charge',
 'Total intl minutes',
 'Total intl calls',
 'Total intl charge',
 'Customer service calls',
 'Churn']

In [10]:
X = data.drop(['Churn'], axis = 1)

In [11]:
for col in columns[:-1]:
    X[col] = encoder.fit_transform(X[col])
data2 = X
X=X.values

In [12]:
Y = data['Churn']
Y.head()
Y = Y.values

In [13]:
X_selected = selector.fit_transform(X, Y)

In [14]:
selected_feature_indices = selector.get_support(indices=True)

In [15]:
selected_feature_names = [data.columns[i] for i in selected_feature_indices[1:]]
selected_feature_names

['Number vmail messages',
 'Total day minutes',
 'Total day charge',
 'Total eve minutes',
 'Total eve charge',
 'Total night minutes',
 'Total night charge',
 'Total intl charge']

In [16]:
df = data[selected_feature_names]
df = df.values
df

array([[ 25.  , 265.1 ,  45.07, ..., 244.7 ,  11.01,   2.7 ],
       [ 26.  , 161.6 ,  27.47, ..., 254.4 ,  11.45,   3.7 ],
       [  0.  , 243.4 ,  41.38, ..., 162.6 ,   7.32,   3.29],
       ...,
       [  0.  , 231.1 ,  39.29, ..., 191.3 ,   8.61,   2.59],
       [  0.  , 180.8 ,  30.74, ..., 191.9 ,   8.64,   3.81],
       [ 25.  , 234.4 ,  39.85, ..., 241.4 ,  10.86,   3.7 ]])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.2, random_state=42)

In [28]:
lr = LogisticRegression(class_weight = {0:0.5,1:1})

In [29]:
lr.fit(X_train, y_train)

LogisticRegression(class_weight={0: 0.5, 1: 1})

In [30]:
y_pred = lr.predict(X_test)

In [34]:
y_prob = lr.predict_proba(X_test)[:, 1]
threshold = 0.47
y_pred_adjusted = [1 if prob >= threshold else 0 for prob in y_prob]

In [35]:
accuracy = accuracy_score(y_test, y_pred_adjusted)
print("Accuracy:", accuracy)

Accuracy: 0.8838951310861424
